In [1]:
# Import needed stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from config import census_api_key
from census import Census

In [2]:
# Get the data from the MDE data available on MDE's website
MDE_data = pd.read_csv("MDE_Data.csv")
MDE_data

,District Number,District Name,Total Enrollment,Total Students of Color or American Indian Count,Total Students of Color or American Indian Percent,Total Students Receiving Special Education Services Count,Total Students Receiving Special Education Services Percent
0,2396,A.C.G.C. Public School District,817,86,10.53%,166,20.32%
1,2854,Ada-Borup Public School District,709,148,20.87%,136,19.18%
2,511,Adrian Public School District,574,109,18.99%,88,15.33%
3,1,Aitkin Public School District,1186,84,7.08%,208,17.54%
4,745,Albany Public School District,1768,76,4.30%,293,16.57%
...,...,...,...,...,...,...,...
328,861,Winona Area Public School District,2937,593,20.19%,656,22.34%
329,518,Worthington Public School District,3904,2569,65.80%,522,13.37%
330,100,Wrenshall Public School District,377,35,9.28%,70,18.57%
331,2190,Yellow Medicine East,696,214,30.75%,155,22.27%


In [4]:
census_response = []

# Get the census population and poverty data for school districts in minnesota
url = f"https://api.census.gov/data/2018/acs/acs5?get=NAME,B01003_001E,B17001_002E&for=school%20district%20(unified):*&in=state:27&key={census_api_key}"
try:
    census_response = requests.get(url).json()
except:
    pass

# Put results in a data from and drop the first row with headers
census_response_df = pd.DataFrame(census_response)
census_response_df = census_response_df.drop([0])

# Rename the columns
census_response_df = census_response_df.rename(columns = {0:"School District", 1:"Population", 2:"Poverty", 3: "State Number", 4:"Census School Number"})

# Strip off Minnesota from school district names so we can merge with MDE data
census_response_df["School District"] = census_response_df["School District"].str.replace(', Minnesota', '')

# Just pull the data we are interested in
clean_census_data = census_response_df.loc[:,["School District", "Population", "Poverty", "Census School Number"]]
clean_census_data.to_csv("Census_data.csv", index=False)
clean_census_data

,School District,Population,Poverty,Census School Number
1,Nevis Public School District,2640,100,23370
2,New London-Spicer School District,9275,441,23400
3,New Prague Area Schools,20274,914,23430
4,New Ulm Public School District,19140,1187,23490
5,New York Mills Public School District,3782,386,23520
...,...,...,...,...
320,Mora Public School District,10603,1139,21480
321,Westonka Public School District,19154,776,22920
322,Mounds View Public School District,79439,5439,22950
323,Mountain Lake Public Schools,2993,398,23010


In [5]:
# Merge the two data frames on the school district name.
merged_census_MDE_df = pd.merge(MDE_data, clean_census_data, right_on = 'School District', left_on = 'District Name')
merged_census_MDE_df.to_csv("Merged_MDE_Census_data.csv", index=False)
merged_census_MDE_df

,District Number,District Name,Total Enrollment,Total Students of Color or American Indian Count,Total Students of Color or American Indian Percent,Total Students Receiving Special Education Services Count,Total Students Receiving Special Education Services Percent,School District,Population,Poverty,Census School Number
0,2396,A.C.G.C. Public School District,817,86,10.53%,166,20.32%,A.C.G.C. Public School District,5663,615,00106
1,2854,Ada-Borup Public School District,709,148,20.87%,136,19.18%,Ada-Borup Public School District,2622,222,00126
2,511,Adrian Public School District,574,109,18.99%,88,15.33%,Adrian Public School District,2275,174,02730
3,1,Aitkin Public School District,1186,84,7.08%,208,17.54%,Aitkin Public School District,9541,1098,02760
4,745,Albany Public School District,1768,76,4.30%,293,16.57%,Albany Public School District,9531,556,02930
...,...,...,...,...,...,...,...,...,...,...,...
315,861,Winona Area Public School District,2937,593,20.19%,656,22.34%,Winona Area Public School District,39562,5831,44070
316,518,Worthington Public School District,3904,2569,65.80%,522,13.37%,Worthington Public School District,16278,2599,44160
317,100,Wrenshall Public School District,377,35,9.28%,70,18.57%,Wrenshall Public School District,1872,140,44190
318,2190,Yellow Medicine East,696,214,30.75%,155,22.27%,Yellow Medicine East,6610,955,00099


In [ ]:
# Part II Start

In [ ]:
# Part II End

In [ ]:
# Part III Start

In [ ]:
# Part III End

In [ ]:
# Part IV Start

In [ ]:
# Part IV End